In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import json

# Paths
dataset_path = '/content/drive/MyDrive/dataset_deepfake'  # Directory where the DFDC videos are stored
output_path = '/content/drive/MyDrive/dataset_deepfake'  # Directory to save extracted frames
metadata_path = os.path.join(dataset_path, 'metadata.json')  # Path to metadata.json

# Create output directories for real and fake frames
real_dir = os.path.join(output_path, 'real')
fake_dir = os.path.join(output_path, 'fake')
os.makedirs(real_dir, exist_ok=True)
os.makedirs(fake_dir, exist_ok=True)

# Function to extract frames from a video
def extract_frames(video_path, output_dir, frame_rate=30):
    video = cv2.VideoCapture(video_path)
    success, frame = video.read()
    count = 0
    while success:
        if count % frame_rate == 0:  # Save one frame per second (assuming 30 FPS)
            frame_filename = os.path.join(output_dir, f"{os.path.basename(video_path)}_frame{count}.jpg")
            cv2.imwrite(frame_filename, frame)
        success, frame = video.read()
        count += 1
    video.release()

# Load metadata to check video labels (real or fake)
with open(metadata_path, 'r') as f:
    metadata = json.load(f)

# Iterate through each video and extract frames based on its label (real or fake)
for video_file, info in metadata.items():
    video_path = os.path.join(dataset_path, video_file)

    # Check if the video is real or fake
    if info['label'] == 'REAL':
        extract_frames(video_path, real_dir)
        print(f"Extracted frames from real video: {video_file}")
    else:
        extract_frames(video_path, fake_dir)
        print(f"Extracted frames from fake video: {video_file}")

print("Frame extraction and organization completed!")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/dataset_deepfake/metadata.json'

In [ ]:
!pip install tensorflow opencv-python matplotlib

In [ ]:
!pip install streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 114.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.3 MB/s eta 0:00:00


In [ ]:
from pyngrok import ngrok
!ngrok authtoken 2qQQPIx0ArL5QbJUhViE7kSBW6p_65mmxw1sHjrrsktYNe9En

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input
from tensorflow.keras.optimizers import Adam

# Set paths to your dataset
dataset_path = "/content/drive/MyDrive/dataset_deepfake"

# Image data generators with augmentation
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Load the Xception model with pre-trained weights from ImageNet
base_model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
base_model.trainable = False

# Input layer
input_tensor = Input(shape=(224, 224, 3))
x = base_model(input_tensor, training=False)  # Set training=False to avoid updating weights

# Flatten and add dense layers
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output_tensor = Dense(1, activation='sigmoid')(x)  # Binary classification (real or deepfake)

# Build the model
model = Model(inputs=input_tensor, outputs=output_tensor)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=5,
    validation_data=validation_generator
)

# Unfreeze base layers for fine-tuning
base_model.trainable = True

# Fine-tune the model with a smaller learning rate
model.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])

history_fine = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

# Save the model
model.save('deepfake_detector.h5')

# Plot accuracy and loss curves
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs_range = range(len(acc))

    plt.figure(figsize=(12, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()

plot_history(history)

Found 0 images belonging to 2 classes.
Found 0 images belonging to 2 classes.
83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


ValueError: The PyDataset has length 0

In [ ]:
%%writefile app.py
import streamlit as st
from tensorflow.keras.models import load_model
import numpy as np
import cv2
from PIL import Image

# Load the trained model
@st.cache_resource
def load_deepfake_model():
    model = load_model('deepfake_detector.h5')  # Update the model path if needed
    return model

# Preprocess the image to match the model input
def preprocess_image(image):
    img = np.array(image)
    img = cv2.resize(img, (224, 224))  # Resize to (224, 224)
    img = img.astype('float32') / 255.0  # Normalize pixel values
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Predict whether the image is real or a deepfake
def predict_image(model, image):
    processed_img = preprocess_image(image)
    prediction = model.predict(processed_img)[0][0]
    return prediction

# Streamlit app UI
def main():
    st.title("Deepfake Image Detection")
    st.write("Upload an image to check if it is real or deepfake.")

    model = load_deepfake_model()

    uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

    if uploaded_file is not None:
        image = Image.open(uploaded_file)
        st.image(image, caption="Uploaded Image", use_column_width=True)

        if st.button("Predict"):
            st.write("Analyzing the image...")
            prediction = predict_image(model, image)

            if prediction > 0.5:
                st.error("The image is a Deepfake.")
            else:
                st.success("The image is Real.")

if __name__ == "__main__":
    main()


Overwriting app.py


In [ ]:
from pyngrok import ngrok

# Set up a tunnel to the streamlit app
public_url = ngrok.connect(8501, bind_tls=True)
print('Public URL:', public_url)

# Run the streamlit app
!streamlit run app.py &

Public URL: NgrokTunnel: "https://814f-34-16-173-132.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.173.132:8501

2024-12-19 07:59:08.197956: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-19 07:59:08.221010: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-19 07:59:08.227851: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-19 07:59:08.243954: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorF